In [1]:
# imports
import pandas as pd
import numpy as np
import time

In [3]:
# reading in csv
outcomes_df = pd.read_csv('../cleanedSources/outcomesCleaned.csv')

In [4]:
# examining df
outcomes_df.head()

,id,name_outcome,dateTime_outcome,dob,outcome_type,outcome_subtype,animal_type,sex_outcome,age_outcome,breed,colour
0,A794011,Chunk,2019-05-08 18:20:00,2017-05-02 00:00:00,Rto-Adopt,Unspecified,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White
1,A776359,Gizmo,2018-07-18 16:02:00,2017-07-12 00:00:00,Adoption,Unspecified,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown
2,A821648,Unknown,2020-08-16 11:38:00,2019-08-16 00:00:00,Euthanasia,Unspecified,Other,Unknown,1 year,Raccoon,Gray
3,A720371,Moose,2016-02-13 17:59:00,2015-10-08 00:00:00,Adoption,Unspecified,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
4,A674754,Unknown,2014-03-18 11:47:00,2014-03-12 00:00:00,Transfer,Partner,Cat,Intact Male,6 days,Domestic Shorthair Mix,Orange Tabby


In [6]:
# observing ages present in df
outcomes_df.groupby(['age_outcome']).count()

,id,name_outcome,dateTime_outcome,dob,outcome_type,outcome_subtype,animal_type,sex_outcome,breed,colour
age_outcome,,,,,,,,,,
-1 years,5,5,5,5,5,5,5,5,5,5
-2 years,1,1,1,1,1,1,1,1,1,1
-3 years,1,1,1,1,1,1,1,1,1,1
0 years,184,184,184,184,184,184,184,184,184,184
1 day,264,264,264,264,264,264,264,264,264,264
1 month,5347,5347,5347,5347,5347,5347,5347,5347,5347,5347
1 week,660,660,660,660,660,660,660,660,660,660
1 weeks,850,850,850,850,850,850,850,850,850,850
1 year,22126,22126,22126,22126,22126,22126,22126,22126,22126,22126


In [7]:
# removing negative values, assuming the negatives are transcription errors
for x in range (0, len(outcomes_df)):
    if outcomes_df.iloc[x, 8][0] == '-':
        outcomes_df.iloc[x, 8] = outcomes_df.iloc[x, 8][1:]

In [8]:
# creating ages in months and years for better consistency using a function and vectorisation
def norm_age(col, newcol1, newcol2):
    months_list = []
    years_list = []
    # for loop to iterate through df
    for x in range (0, len(outcomes_df)):
        # split each value in the age_intake col by spaces
        # if it ends with year/years, multiply by 12 and enter into month col; enter number into year col
        if (outcomes_df.iloc[x, col].split(' ')[1] == 'year') or (outcomes_df.iloc[x, col].split(' ')[1] == 'years'):
            age_val = int(outcomes_df.iloc[x, col].split(' ')[0])
            months_list.append(round(age_val * 12, 2))
            years_list.append(round(age_val, 2))
        # if it ends with month/months, enter number into month col; divide by 12 and place into year col
        elif (outcomes_df.iloc[x, col].split(' ')[1] == 'month') or (outcomes_df.iloc[x, col].split(' ')[1] == 'months'):
            age_val = int(outcomes_df.iloc[x, col].split(' ')[0])
            months_list.append(round(age_val, 2))
            years_list.append(round(age_val/12, 2))
        # if it ends with week/weeks, divide by four and place into month col; leave default value in year col
        elif (outcomes_df.iloc[x, col].split(' ')[1] == 'week') or (outcomes_df.iloc[x, col].split(' ')[1] == 'weeks'):
            age_val = int(outcomes_df.iloc[x, col].split(' ')[0])
            months_list.append(round(age_val/4, 2))
            years_list.append(0)
        # else (if it ends with day or days), leave default value in both cols
        else:
            months_list.append(0)
            years_list.append(0)
    outcomes_df[f'{newcol1}'] = months_list
    outcomes_df[f'{newcol2}'] = years_list

In [9]:
# vectorising and running the funct (using different col names to examine side by side)
vectfunc = np.vectorize(norm_age)
vectfunc(8, 'age_outcome_months', 'age_outcome_years')
outcomes_df.head()

,id,name_outcome,dateTime_outcome,dob,outcome_type,outcome_subtype,animal_type,sex_outcome,age_outcome,breed,colour,age_outcome_months,age_outcome_years
0,A794011,Chunk,2019-05-08 18:20:00,2017-05-02 00:00:00,Rto-Adopt,Unspecified,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White,24.0,2.00
1,A776359,Gizmo,2018-07-18 16:02:00,2017-07-12 00:00:00,Adoption,Unspecified,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown,12.0,1.00
2,A821648,Unknown,2020-08-16 11:38:00,2019-08-16 00:00:00,Euthanasia,Unspecified,Other,Unknown,1 year,Raccoon,Gray,12.0,1.00
3,A720371,Moose,2016-02-13 17:59:00,2015-10-08 00:00:00,Adoption,Unspecified,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff,4.0,0.33
4,A674754,Unknown,2014-03-18 11:47:00,2014-03-12 00:00:00,Transfer,Partner,Cat,Intact Male,6 days,Domestic Shorthair Mix,Orange Tabby,0.0,0.00


In [10]:
# re-observing ages present in df (months)
outcomes_df.groupby(['age_outcome_months']).count()

,id,name_outcome,dateTime_outcome,dob,outcome_type,outcome_subtype,animal_type,sex_outcome,age_outcome,breed,colour,age_outcome_years
age_outcome_months,,,,,,,,,,,,
0.00,1779,1779,1779,1779,1779,1779,1779,1779,1779,1779,1779,1779
0.25,1510,1510,1510,1510,1510,1510,1510,1510,1510,1510,1510,1510
0.50,2033,2033,2033,2033,2033,2033,2033,2033,2033,2033,2033,2033
0.75,2124,2124,2124,2124,2124,2124,2124,2124,2124,2124,2124,2124
1.00,7149,7149,7149,7149,7149,7149,7149,7149,7149,7149,7149,7149
1.25,121,121,121,121,121,121,121,121,121,121,121,121
2.00,15028,15028,15028,15028,15028,15028,15028,15028,15028,15028,15028,15028
3.00,5943,5943,5943,5943,5943,5943,5943,5943,5943,5943,5943,5943
4.00,4023,4023,4023,4023,4023,4023,4023,4023,4023,4023,4023,4023


In [11]:
# re-observing ages present in df (years)
outcomes_df.groupby(['age_outcome_years']).count()

,id,name_outcome,dateTime_outcome,dob,outcome_type,outcome_subtype,animal_type,sex_outcome,age_outcome,breed,colour,age_outcome_months
age_outcome_years,,,,,,,,,,,,
0.00,9369,9369,9369,9369,9369,9369,9369,9369,9369,9369,9369,9369
0.08,5347,5347,5347,5347,5347,5347,5347,5347,5347,5347,5347,5347
0.17,15028,15028,15028,15028,15028,15028,15028,15028,15028,15028,15028,15028
0.25,5943,5943,5943,5943,5943,5943,5943,5943,5943,5943,5943,5943
0.33,4023,4023,4023,4023,4023,4023,4023,4023,4023,4023,4023,4023
0.42,3098,3098,3098,3098,3098,3098,3098,3098,3098,3098,3098,3098
0.50,2996,2996,2996,2996,2996,2996,2996,2996,2996,2996,2996,2996
0.58,1598,1598,1598,1598,1598,1598,1598,1598,1598,1598,1598,1598
0.67,1981,1981,1981,1981,1981,1981,1981,1981,1981,1981,1981,1981


In [12]:
# re-saving to csv
outcomes_df.to_csv('../cleanedSources/outcomesCleaned.csv', index=False)

In [13]:
# checking time it takes to run the vector function
%timeit vectfunc(8, 'age_outcome_months', 'age_outcome_years')

26.6 s ± 1.52 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
# trying age regularisation through for loop for time comparison
t1=time.time()
months_list = []
years_list = []
# for loop to iterate through df
for x in range (0, len(outcomes_df)):
    # split each value in the age_intake col by spaces
    # if it ends with year/years, multiply by 12 and enter into month col; enter number into year col
    if (outcomes_df.iloc[x, 8].split(' ')[1] == 'year') or (outcomes_df.iloc[x, 8].split(' ')[1] == 'years'):
        age_val = int(outcomes_df.iloc[x, 8].split(' ')[0])
        months_list.append(age_val * 12)
        years_list.append(age_val)
    # if it ends with month/months, enter number into month col; divide by 12 and place into year col
    elif (outcomes_df.iloc[x, 8].split(' ')[1] == 'month') or (outcomes_df.iloc[x, 8].split(' ')[1] == 'months'):
        age_val = int(outcomes_df.iloc[x, 8].split(' ')[0])
        months_list.append(age_val)
        years_list.append(age_val/12)
    # if it ends with week/weeks, divide by four and place into month col; leave default value in year col
    elif (outcomes_df.iloc[x, 8].split(' ')[1] == 'week') or (outcomes_df.iloc[x, 8].split(' ')[1] == 'weeks'):
        age_val = int(outcomes_df.iloc[x, 8].split(' ')[0])
        months_list.append(age_val/4)
        years_list.append(0)
    # else (if it ends with day or days), leave default value in both cols
    else:
        months_list.append(0)
        years_list.append(0)
t2=time.time()
print("\nTime taken by for-loop\n----------------------------------------------\n{} s".format(t2-t1))


Time taken by for-loop
----------------------------------------------
13.335582971572876 s
